In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc

from IPython.display import display

/public/workspace/licanchengup/apps/miniconda3/envs/csMAHN/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
p_link = Path('/public/workspace/licanchengup/link')
p_test = p_link.joinpath('test')
p_root = Path('/public/workspace/licanchengup/link/db/came/dataset/Retina')
None if str(p_test) in sys.path else sys.path.append(str(p_test))

In [3]:
from func import h5ad_to_mtx, group_agg


[func]----------------------------------
group_agg
rm_rf
get_path_varmap
get_type_counts_info		aligned
aligned_type			aligned
test_came_and_csMAHN
h5ad_to_mtx
load_adata

[result tools]--------------------------
get_test_result_df
show_umap
get_res_obs
get_publi_head

[result tools][re plot umap]-----------
get_df_color			Convenient
> [dssp umap]
export_legend
dssp_plot_umap_legend_sp	Convenient
dssp_plot_umap_legend_cell_type	Convenient
dssp_plot_umap
dssp_get_umap_name
dssp_plot_save_umap		Convenient
> [umap]
get_default_df_color
plot_umap
get_umap_name
plot_save_umap			Convenient
----------------------------------------
    


In [4]:
# df_path = pd.DataFrame({
#     'mtx': [Path('Single-cell-retinal-regeneration-master/Zebrafish_{}_count_matrix.mtx'.format(_))
#             for _ in 'LD,NMDA,TNFa'.split(',')],
#     'obs': [Path('Single-cell-retinal-regeneration-master/Zebrafish_{}_cell_features.tsv'.format(_))
#             for _ in 'LD,NMDA,TNFa'.split(',')]

# }, index='LD,NMDA,TNFa'.split(','))
# display(df_path.head(2))

# df_var = pd.read_csv(
#     Path('Single-cell-retinal-regeneration-master/Zebrafish_gene_features.tsv'),
#     sep='\t',
#     index_col=1)
# display(df_var.head(2), df_var.shape, df_var.index.is_unique)

# for _i, _row in df_path.iterrows():
#     print(
#         ','.join(
#             pd.read_csv(
#                 _row['obs'],
#                 sep='\t',
#                 index_col=0).columns))

# _row


_row = {
    'mtx': Path('Single-cell-retinal-regeneration-master/Chick_NMDAandGF_count_matrix.mtx'),
    'obs': Path('Single-cell-retinal-regeneration-master/Chick_NMDAandGF_cell_features.tsv'),

}
_row

{'mtx': PosixPath('Single-cell-retinal-regeneration-master/Chick_NMDAandGF_count_matrix.mtx'),
 'obs': PosixPath('Single-cell-retinal-regeneration-master/Chick_NMDAandGF_cell_features.tsv')}

In [5]:
df_var = pd.read_csv(
    Path('Single-cell-retinal-regeneration-master/Chick_gene_features.tsv'),
    sep='\t',
    index_col=None,
    header=0,
    names='gene_id,gene_name'.split(','))

display(df_var['gene_name'].isna().value_counts())
df_var['gene_name'] = df_var['gene_name'].mask(
    df_var['gene_name'].isna(), df_var['gene_id'])
display(df_var['gene_name'].isna().value_counts())
df_var.index = df_var['gene_name'].to_numpy()
display(df_var.head(2), df_var.shape, df_var.index.is_unique)

gene_name
False    24654
True        24
Name: count, dtype: int64

gene_name
False    24678
Name: count, dtype: int64

,gene_id,gene_name
ENSGALG00000045540,ENSGALG00000045540,ENSGALG00000045540
LOC430443,ENSGALG00000031626,LOC430443


(24678, 2)

False

In [6]:
# res = {}
# for _i, _row in df_path.iterrows():
#     pass
#     print(_row.name.ljust(50, '-'))

adata = sc.read_mtx(_row['mtx']).T

df_obs = pd.read_csv(_row['obs'],
                     sep='\t',
                     index_col=0).loc[:,
                                      'Sample,Cell.type,tSNE.1,tSNE.2'.split(',')]
df_obs = df_obs['Sample'].str.extract(
    '^(?P<age>[^ ]+) (?P<tissue>[^ ]+)').join(df_obs)
df_obs = df_obs.rename(columns={'Sample': '_batch'})
display(
    df_obs.head(2),
    df_obs.shape,
    df_obs['_batch'].value_counts(),
    df_obs['Cell.type'].value_counts())
adata.var = df_var
adata.obs = df_obs
display(adata.obs.index.is_unique, adata.var.index.is_unique)
adata.var_names_make_unique()
adata.obs_names_make_unique()
display(
    adata,
    adata.obs.head(2),
    adata.obs.shape,
    adata.var.head(2),
    adata.var.shape)
# res.update({_row.name: adata})
# del adata

,age,tissue,_batch,Cell.type,tSNE.1,tSNE.2
#Barcode,,,,,,
AAACCTGCAGCGTCCA-chP10R1,P10,R1,P10 R1,Resting MG,10.044577,30.207655
AAACCTGTCTCAACTT-chP10R1,P10,R1,P10 R1,Resting MG,9.477344,30.907544


(80777, 6)

_batch
P10 R2             11781
72hr NMDA R1        9922
48hr GF R2          9554
72hr NMDA R2        8195
24hr NMDA R2        7726
48hr GF R1          7389
48hr NMDA R2        6724
48hr NMDA+GF R1     6055
24hr NMDA R1        5113
48hr NMDA+GF R2     4856
P10 R1              2038
48hr NMDA R1        1424
Name: count, dtype: int64

Cell.type
Activated MG        59555
Resting MG           5293
Cone BC              3726
GABAergic AC         3712
RGC                  2400
Oligodendrocytes     1874
NIRG                 1685
Cones                1635
Rods                  897
Name: count, dtype: int64

True

...

AnnData object with n_obs × n_vars = 80777 × 24678
    obs: 'age', 'tissue', '_batch', 'Cell.type', 'tSNE.1', 'tSNE.2'
    var: 'gene_id', 'gene_name'

,age,tissue,_batch,Cell.type,tSNE.1,tSNE.2
#Barcode,,,,,,
AAACCTGCAGCGTCCA-chP10R1,P10,R1,P10 R1,Resting MG,10.044577,30.207655
AAACCTGTCTCAACTT-chP10R1,P10,R1,P10 R1,Resting MG,9.477344,30.907544


(80777, 6)

,gene_id,gene_name
ENSGALG00000045540,ENSGALG00000045540,ENSGALG00000045540
LOC430443,ENSGALG00000031626,LOC430443


(24678, 2)

In [7]:
adata.obs['pre_ct'] = adata.obs['Cell.type']
adata.obs['cell_type'] = adata.obs['Cell.type'].map(lambda k: {
    'GABAergic AC': 'AC',

    'Resting MG': 'Macroglial cell',
    'Activated MG': 'Macroglial cell',
    'Oligodendrocytes': 'Macroglial cell',

    'Cone BC': 'BC',
    'Rods': 'Rods',
    'Cones': 'Cones',

    'RGC': 'RGC',
    'NIRG': 'chick_P10_NIRG'}.setdefault(k, k))
for _ in 'age,tissue,_batch,Cell.type,pre_ct,cell_type'.split(','):
    print('\n', _.ljust(50, '-'))
    display(adata.obs[_].value_counts())


 age-----------------------------------------------


age
48hr    36002
72hr    18117
P10     13819
24hr    12839
Name: count, dtype: int64


 tissue--------------------------------------------


tissue
NMDA       39104
GF         16943
R2         11781
NMDA+GF    10911
R1          2038
Name: count, dtype: int64


 _batch--------------------------------------------


...

Cell.type
Activated MG        59555
Resting MG           5293
Cone BC              3726
GABAergic AC         3712
RGC                  2400
Oligodendrocytes     1874
NIRG                 1685
Cones                1635
Rods                  897
Name: count, dtype: int64


 pre_ct--------------------------------------------


pre_ct
Activated MG        59555
Resting MG           5293
Cone BC              3726
GABAergic AC         3712
RGC                  2400
Oligodendrocytes     1874
NIRG                 1685
Cones                1635
Rods                  897
Name: count, dtype: int64


 cell_type-----------------------------------------


cell_type
Macroglial cell    66722
BC                  3726
AC                  3712
RGC                 2400
chick_P10_NIRG      1685
Cones               1635
Rods                 897
Name: count, dtype: int64

In [8]:
_ = adata[adata.obs['age'] == 'P10',:].copy()
display(_.obs['age'].value_counts())
display(_.obs.head(2),_.obs.shape, _.obs.index.is_unique, _.var.index.is_unique)
h5ad_to_mtx(_, p_root.joinpath('chicken_P10'))

age
P10    13819
Name: count, dtype: int64

,age,tissue,_batch,Cell.type,tSNE.1,tSNE.2,pre_ct,cell_type
#Barcode,,,,,,,,
AAACCTGCAGCGTCCA-chP10R1,P10,R1,P10 R1,Resting MG,10.044577,30.207655,Resting MG,Macroglial cell
AAACCTGTCTCAACTT-chP10R1,P10,R1,P10 R1,Resting MG,9.477344,30.907544,Resting MG,Macroglial cell


(13819, 8)

True

True

frist 10 data.X nonzero elements:
 [[1 1 2 1 2 1 1 2 1 2]]
[out] /public/workspace/licanchengup/link/db/came/dataset/Retina/chicken_P10


||||
|:-|:-|:-|
|AC| amacrine cells|无长突细胞；无轴突细胞；无突起细胞|
|V/E cell| vascular/endothelial cells|血管/内皮细胞|
|Macroglial cell||大胶质细胞|
|Microglia cell||小胶质细胞|
|Pericyte cell||周细胞|
|Progenitor cell||祖细胞|
|HC|||
|BC| bipolar cells|双极细胞；双极性细胞|
|Cones||视锥细胞|
|RGC| retinal ganglion cells|视网膜神经节细胞|
|RPE| retinal pigment epithelium|视网膜色素上皮细胞|
|Rods||视杆细胞|
|NIRG| nonastrocytic inner retinal glial cells|视网膜内非星形胶质细胞|

In [9]:
for x in _.obs['Cell.type'].unique():
    print("'{}':'',".format(x))

'Resting MG':'',
'Cone BC':'',
'Rods':'',
'Oligodendrocytes':'',
'GABAergic AC':'',
'RGC':'',
'Cones':'',
'Activated MG':'',
'NIRG':'',
